In [46]:
import pandas as pd
from itertools import product,combinations
def simulate_round_all(df, round_num):
    previous_rounds_df = df[df["Round"] < f"Round {round_num}"]
    ranking_df = calculate_ranking(df, round_num - 1)
    # Exclude teams with 3 wins or losses
    ranking_df = ranking_df[(ranking_df["Wins"] < 3) & (ranking_df["Losses"] < 3)]
    groups_by_wins = ranking_df.groupby("Wins")["Team"].apply(list).to_dict()
    matchups = []
    for teams in groups_by_wins.values():
        matchups.extend(list(combinations(teams, 2)))
    matchups_df = pd.DataFrame(matchups, columns=["T1", "T2"])
    matchups_df["Score T1"] = 0
    matchups_df["Score T2"] = 0
    matchups_df["Round"] = f"Round {round_num}"
    return matchups_df

def add_match_result(df, t1, t2, score_t1, score_t2, round_num):
    new_row = {
        'T1': t1,
        'T2': t2,
        'Score T1': score_t1,
        'Score T2': score_t2,
        'Round': round_num
    }
    if check_repeat_matchup(df, t1, t2):
        df.loc[(df['T1'] == t1) & (df['T2'] == t2), ['Score T1', 'Score T2']] = [score_t1, score_t2]
    else:
        df = df.append(new_row, ignore_index=True)
    return df

def team_potential_draw(df, team):
    df_temp = pd.concat([df[df["T1"] == team], df[df["T2"] == team]])
    return df_temp

In [23]:
def calculate_round5_probabilities(df, round_num):
    """
    Calculate the probability for each team to need to play in Round 5 after Round 4.

    Args:
    - df: DataFrame containing match results up to Round 3.
    - round_num: The current round number (should be 4).

    Returns:
    - A DataFrame with teams and their probabilities to play in Round 5.
    """
    # Get current standings after Round 3
    current_ranking = calculate_ranking(df, round_num - 1)
    teams = current_ranking["Team"].tolist()
    team_records = current_ranking.set_index("Team")[["Wins", "Losses"]].to_dict("index")
    
    # Get Round 4 matchups
    matchups = round4_df[["T1", "T2"]].values.tolist()
    
    # Total possible outcomes
    total_outcomes = 2 ** len(matchups)
    
    # Initialize a dictionary to count the number of times each team needs to play in Round 5
    round5_counts = {team: 0 for team in teams}
    
    # Generate all possible outcomes
    outcome_list = list(product([0, 1], repeat=len(matchups)))
    
    for outcome in outcome_list:
        # Copy team records for this scenario
        scenario_records = {team: rec.copy() for team, rec in team_records.items()}
        
        # Apply the outcomes to the matchups
        for idx, result in enumerate(outcome):
            t1, t2 = matchups[idx]
            # If result == 1, t1 wins; else t2 wins
            if result == 1:
                scenario_records[t1]["Wins"] += 1
                scenario_records[t2]["Losses"] += 1
            else:
                scenario_records[t1]["Losses"] += 1
                scenario_records[t2]["Wins"] += 1
        
        # Determine status of each team
        for team in teams:
            wins = scenario_records[team]["Wins"]
            losses = scenario_records[team]["Losses"]
            if wins < 3 and losses < 3:
                round5_counts[team] += 1
    
    # Calculate probabilities
    probabilities = {team: count / total_outcomes for team, count in round5_counts.items()}
    
    # Prepare the result DataFrame
    prob_df = pd.DataFrame(list(probabilities.items()), columns=["Team", "Prob_Round5"])
    prob_df = prob_df.sort_values(by="Prob_Round5", ascending=False).reset_index(drop=True)
    
    return prob_df
def calculate_ranking(df, round_num):
    # Filter matches up to the specified round
    filtered_df = df[df["Round"] <= f"Round {round_num}"]
    # Initialize ranking dictionary
    ranking = {}
    # Calculate wins, losses, games played
    for _, row in filtered_df.iterrows():
        t1, t2 = row['T1'], row['T2']
        score_t1, score_t2 = row['Score T1'], row['Score T2']
        for team in [t1, t2]:
            if team not in ranking:
                ranking[team] = {"Wins": 0, "Losses": 0, "Games Played": 0}
        if score_t1 > score_t2:
            ranking[t1]["Wins"] += 1
            ranking[t2]["Losses"] += 1
        elif score_t1 != score_t2:
            ranking[t1]["Losses"] += 1
            ranking[t2]["Wins"] += 1
        ranking[t1]["Games Played"] += 1
        ranking[t2]["Games Played"] += 1
    # Convert to DataFrame
    ranking_df = pd.DataFrame.from_dict(ranking, orient='index').reset_index()
    ranking_df.columns = ["Team", "Wins", "Losses", "Games Played"]
    # Sort by Wins and Games Played
    ranking_df = ranking_df.sort_values(by=["Wins", "Games Played"], ascending=[False, True]).reset_index(drop=True)
    # Add Rank column
    ranking_df['Rank'] = ranking_df.index + 1
    return ranking_df[["Rank", "Team", "Wins", "Losses", "Games Played"]]



In [6]:
df=pd.read_csv(r"C:\Users\Aymen\Documents\Projects\Lol swiss\All.csv")

In [12]:
df["T1"]=df["T1"].str.replace("Bilibili Gaming","BLG")
df["T2"]=df["T2"].str.replace("Bilibili Gaming","BLG")

In [24]:
calculate_ranking(df,3)

,Rank,Team,Wins,Losses,Games Played
0,1,Gen.G,3,0,3
1,2,LNG Esports,3,0,3
2,3,Dplus KIA,2,1,3
3,4,Hanwha Life Esports,2,1,3
4,5,BLG,1,1,3
5,6,Top Esports,1,1,3
6,7,T1,1,1,3
7,8,Weibo Gaming,1,1,3
8,9,Fnatic,1,1,3
9,10,PSG Talon,1,1,3


In [22]:
df

,Match,T1,Score T1,Score T2,T2,Round
0,BLG vs MAD Lions,BLG,1,0,MAD Lions,Round 1
1,Top Esports vs T1,Top Esports,1,0,T1,Round 1
2,Gen.G vs Weibo Gaming,Gen.G,1,0,Weibo Gaming,Round 1
3,Fnatic vs Dplus KIA,Fnatic,0,1,Dplus KIA,Round 1
4,Team Liquid vs LNG Esports,Team Liquid,0,1,LNG Esports,Round 1
5,Hanwha Life Esports vs PSG Talon,Hanwha Life Esports,1,0,PSG Talon,Round 1
6,FlyQuest vs GAM Esports,FlyQuest,1,0,GAM Esports,Round 1
7,G2 Esports vs PaiN Gaming,G2 Esports,1,0,PaiN Gaming,Round 1
8,BLG vs LNG Esports,BLG,0,1,LNG Esports,Round 2
9,Gen.G vs Top Esports,Gen.G,1,0,Top Esports,Round 2


In [27]:
def add_match_result(df, t1, t2, score_t1, score_t2, round_num):
    new_row = {
        'T1': t1,
        'T2': t2,
        'Score T1': score_t1,
        'Score T2': score_t2,
        'Round': round_num
    }
    if check_repeat_matchup(df, t1, t2):
        df.loc[(df['T1'] == t1) & (df['T2'] == t2), ['Score T1', 'Score T2']] = [score_t1, score_t2]
    else:
        df = df.append(new_row, ignore_index=True)
    return df

def team_potential_draw(df, team):
    df_temp = pd.concat([df[df["T1"] == team], df[df["T2"] == team]])
    return df_temp

def check_repeat_matchup(df, team1, team2):
    condition_1 = (df['T1'] == team1) & (df['T2'] == team2)
    condition_2 = (df['T1'] == team2) & (df['T2'] == team1)
    return df[condition_1 | condition_2].shape[0] > 0

In [92]:
xss=simulate_round_all(df,4)

In [93]:
repeat_columns = []
for index,row in xss.iterrows():
    repeat_columns.append(check_repeat_matchup(df,row.loc["T1"],row.loc["T2"]))

In [94]:
xss["Repeat"]=repeat_columns

In [96]:
xss

,T1,T2,Score T1,Score T2,Round,Repeat
0,BLG,MAD Lions,0,0,Round 4,True
1,BLG,Weibo Gaming,0,0,Round 4,False
2,BLG,Fnatic,0,0,Round 4,False
3,BLG,Team Liquid,0,0,Round 4,False
4,BLG,PSG Talon,0,0,Round 4,False
5,MAD Lions,Weibo Gaming,0,0,Round 4,False
6,MAD Lions,Fnatic,0,0,Round 4,False
7,MAD Lions,Team Liquid,0,0,Round 4,False
8,MAD Lions,PSG Talon,0,0,Round 4,True
9,Weibo Gaming,Fnatic,0,0,Round 4,False


In [74]:
type(xss["Repeat"].loc[5])

numpy.bool

In [86]:
team_potential_draw(xss,"T1")

,T1,T2,Score T1,Score T2,Round,Repeat
20,T1,Dplus KIA,0,0,Round 4,False
21,T1,Hanwha Life Esports,0,0,Round 4,False
22,T1,FlyQuest,0,0,Round 4,False
23,T1,G2 Esports,0,0,Round 4,False
15,Top Esports,T1,0,0,Round 4,True


In [83]:
team_potential_draw(xss,"G2 Esports")

,T1,T2,Score T1,Score T2,Round,Repeat
19,Top Esports,G2 Esports,0,0,Round 4,False
23,T1,G2 Esports,0,0,Round 4,False
26,Dplus KIA,G2 Esports,0,0,Round 4,False
28,Hanwha Life Esports,G2 Esports,0,0,Round 4,True
29,FlyQuest,G2 Esports,0,0,Round 4,False


In [87]:
EU=["G2 Esports","Fnatic","MAD Lions"]
NA=["Team Liquid","FlyQuest"]
KR=["Gen.G","T1","Dplus KIA","Hanwha Life Esports"]
CN=["Weibo Gaming","Top Esports","BLG","LNG Esports"]
TW=["PSG Talon"]

,T1,T2,Score T1,Score T2,Round,Repeat
0,BLG,MAD Lions,0,0,Round 4,True
1,BLG,Weibo Gaming,0,0,Round 4,False
2,BLG,Fnatic,0,0,Round 4,False
3,BLG,Team Liquid,0,0,Round 4,False
4,BLG,PSG Talon,0,0,Round 4,False
5,MAD Lions,Weibo Gaming,0,0,Round 4,False
6,MAD Lions,Fnatic,0,0,Round 4,False
7,MAD Lions,Team Liquid,0,0,Round 4,False
8,MAD Lions,PSG Talon,0,0,Round 4,True
9,Weibo Gaming,Fnatic,0,0,Round 4,False


In [107]:
upper_bracket_matchings =sss[sss["T1"].isin(UpperBracket)]


NameError: name 'UpperBracket' is not defined

In [141]:
UpperBracket=["Top Esports","T1","Dplus KIA","Hanwha Life Esports","FlyQuest","G2 Esports"]
LowerBracket=["BLG","MAD Lions","Weibo Gaming","Fnatic","Team Liquid","PSG Talon"]
upper_bracket_matchings =generate_matchings(UpperBracket)
lower_bracket_matchings =generate_matchings(LowerBracket)



In [144]:
def remove_scenario_repeat(df,matchings):    
    index_to_remove=[]
    exist=False
    for i in range(len(matchings)) :
        for j in range(len(matchings[i])):
            if check_repeat_matchup(df,matchings[i][j][0],matchings[i][j][1])  :
                exist=True
        if exist==True :
            index_to_remove.append(i)
        exist=False    
    L_reduced = [element for idx, element in enumerate(matchings) if idx not in index_to_remove]
  
    return L_reduced


In [146]:
upper_bracket_matchings=remove_scenario_repeat(df,upper_bracket_matchings)
lower_bracket_matchings=remove_scenario_repeat(df,lower_bracket_matchings)


In [157]:
def calculate_round5_probabilities(upper_bracket_matchings,lower_bracket_matchings,region_list,regions):

    total_scenarios = 0

    for upper_matchups in upper_bracket_matchings:
        for lower_matchups in lower_bracket_matchings:
            # Combine matchups
            round4_matchups = upper_matchups + lower_matchups
            
            # Generate all possible outcomes for Round 4
            outcomes_round4 = list(product([0, 1], repeat=6))  # 6 matches in total
            
            for outcome4 in outcomes_round4:
                # Determine Round 4 winners and losers
                round4_winners = []
                round4_losers = []
                eliminated = []
                
                for idx, result in enumerate(outcome4):
                    team1, team2 = round4_matchups[idx]
                    if idx < 3:  # Upper Bracket
                        if result == 1:
                            round4_winners.append(team1)
                            round4_losers.append(team2)
                        else:
                            round4_winners.append(team2)
                            round4_losers.append(team1)
                    else:  # Lower Bracket
                        if result == 1:
                            round4_winners.append(team1)
                            eliminated.append(team2)
                        else:
                            round4_winners.append(team2)
                            eliminated.append(team1)
                
                # Generate Round 5 matchups: Losers of Upper Bracket vs Winners of Lower Bracket
                # Need to generate all possible matchings between these teams, considering no repeats and rules
                # Since there are 3 Upper Losers and 3 Lower Winners, possible matchings are:
                # Number of matchings: 15
                
                # For simplicity, we'll assume the matchups are fixed in the order of lists
                round5_matchups = list(zip(round4_losers, round4_winners[3:]))
                
                # Generate all possible outcomes for Round 5
                outcomes_round5 = list(product([0, 1], repeat=3))
                
                for outcome5 in outcomes_round5:
                    total_scenarios += 1
                    advanced_teams = round4_winners[:3]  # Upper Bracket winners
                    eliminated_teams = eliminated.copy()
                    
                    # Simulate Round 5 matches
                    for idx, result in enumerate(outcome5):
                        team1, team2 = round5_matchups[idx]
                        if result == 1:
                            advanced_teams.append(team1)
                            eliminated_teams.append(team2)
                        else:
                            advanced_teams.append(team2)
                            eliminated_teams.append(team1)
                    
                    # Tally the number of teams from each region that advance
                    region_counts = {region: 0 for region in region_list}
                    for team in advanced_teams:
                        for region, teams in regions.items():
                            if team in teams:
                                region_counts[region] += 1
                    
                    # Append the counts to the results
                    for region in region_list:
                        results[region].append(region_counts[region])
    probabilities = {}

    for region in region_list:
        counts = results[region]
        total = len(counts)
        freq = pd.Series(counts).value_counts().sort_index()
        prob = (freq / total).reset_index()
        print(total)
        prob.columns = ['Teams_Advancing', 'Probability']
        probabilities[region] = prob
    return probabilities

In [168]:
regions = {
    "West": ["G2 Esports", "Fnatic", "MAD Lions","Team Liquid", "FlyQuest"],
    "East": ["T1", "Dplus KIA", "Hanwha Life Esports","Weibo Gaming", "Top Esports", "BLG","PSG Talon"]
}
region_list = ["West","East"]
results = {region: [] for region in region_list}

calculate_round5_probabilities(upper_bracket_matchings,lower_bracket_matchings,region_list,regions)


32768
32768


{'West':    Teams_Advancing  Probability
 0                0     0.005127
 1                1     0.132568
 2                2     0.512207
 3                3     0.308105
 4                4     0.041260
 5                5     0.000732,
 'East':    Teams_Advancing  Probability
 0                1     0.000732
 1                2     0.041260
 2                3     0.308105
 3                4     0.512207
 4                5     0.132568
 5                6     0.005127}

In [172]:
from pathlib import Path
p = Path('.')

In [179]:
df.to_csv(Path("../data/matches.csv"))

In [181]:
calculate_ranking(df,4)

,Rank,Team,Wins,Losses,Games Played
0,1,Gen.G,3,0,3
1,2,LNG Esports,3,0,3
2,3,Top Esports,2,1,3
3,4,T1,2,1,3
4,5,Dplus KIA,2,1,3
5,6,Hanwha Life Esports,2,1,3
6,7,FlyQuest,2,1,3
7,8,G2 Esports,2,1,3
8,9,BLG,1,2,3
9,10,MAD Lions,1,2,3


In [161]:
len(lower_bracket_matchings)

8

In [162]:
2**8

256

In [164]:
8*8*8*8

4096

In [167]:
((2**3)*(2**3))*((2**3)*(2**3))*(2**3)

32768

In [ ]:
8*8*